# Taller Integrador – Sistema Experto de Eficiencia Energética(ejemplo)

**Nombre del Archivo a evaluar:** `Taller_1.ipynb`  
**Objetivo:** Sistema experto que detecta **sobreconsumo**, explica **trazabilidad** y propone **acciones**; valida **normalización**.


## Instrucciones*

- Mantén los **nombres** de clases/campos requeridos por el autograder: `Dispositivo`, `Habito`, `Medicion`, `Hipotesis`, `Accion`, `SistemaEnergia`, `declarar_base`, `RESULTADOS`.
- Uso de `salience` y **trazabilidad** (imprime reglas activadas con el prefijo `ACTIVADA:`).
- Escenarios implementados:
  - **A:** AC > umbral + `ac_puertas_abiertas` ⇒ `sobreconsumo_clima` (**crítica**).
  - **B:** Iluminación + `luces_encendidas_sin_uso` > baseline ⇒ `iluminacion_ineficiente` (**media**).
  - **C:** Normalización (consumo vuelve a baseline) ⇒ retractar hipótesis previa y declarar `consumo_normalizado` (**baja**).


## Retirar la linea de codigo !pip install experta antes de subir los cambios

In [ ]:
import collections.abc
if not hasattr(collections, 'Mapping'):
    collections.Mapping = collections.abc.Mapping

In [ ]:
# --- Setup ---
import pandas as pd
from experta import Fact, KnowledgeEngine, Rule, AND, OR, NOT, TEST, MATCH


In [ ]:
# --- Catálogo AUTOCONTENIDO ---
DATA_DEVICES = [
    ("Nevera (Inverter)", 120, 24.0, 2.88, 3.60, "kWh_24h_enchufe_inteligente", "Ajustar 3–5 °C; limpiar sellos; despejar rejillas"),
    ("Aire acondicionado 12000 BTU", 1200, 6.0, 7.20, 9.00, "kWh_6h_intervalo", "Cerrar puertas/ventanas; setpoint 23–25 °C; limpiar filtros"),
    ("Lavadora (ciclo activo)", 500, 1.0, 0.50, 0.63, "kWh_por_ciclo", "Cargas completas; modo eco; fuera de pico"),
    ("Secadora eléctrica", 2000, 0.8, 1.60, 2.00, "kWh_por_ciclo", "Tender al aire; limpiar filtros; cargas completas"),
    ("Horno microondas", 1200, 0.3, 0.36, 0.45, "kWh_por_uso", "Tiempos exactos; cubrir alimentos"),
    ("Televisor LED 43\"", 80, 5.0, 0.40, 0.50, "kWh_diario", "Autoapagado; evitar standby prolongado"),
    ("Computador portátil", 65, 6.0, 0.39, 0.49, "kWh_diario", "Ahorro de energía; desconectar cargador al 80–90%"),
    ("Router WiFi", 10, 24.0, 0.24, 0.30, "kWh_24h", "Programar apagado nocturno si no se usa"),
    ("Iluminación LED x10", 90, 4.0, 0.36, 0.45, "kWh_diario", "Sensores o política de apagado; bombillos 5–7 W"),
    ("Ducha eléctrica/instantáneo", 3500, 0.4, 1.40, 1.75, "kWh_por_uso", "Ducha 5–7 min; regular caudal; aislar tuberías"),
]

DATA_HABITS = [
    ("luces_encendidas_sin_uso", "Dejar luces encendidas en zonas desocupadas.", "Iluminación LED x10"),
    ("standby_tv", "Televisor en modo standby toda la noche.", "Televisor LED 43\""),
    ("ac_puertas_abiertas", "Usar aire acondicionado con puertas/ventanas abiertas.", "Aire acondicionado 12000 BTU"),
    ("nevera_mala_config", "Temperatura de nevera fuera de rango (muy baja).", "Nevera (Inverter)"),
    ("cargas_fuera_pico", "Uso de alta potencia en horas pico.", "Lavadora; Secadora eléctrica; Ducha eléctrica/instantáneo"),
    ("router_24_7", "Router encendido 24/7 sin necesidad.", "Router WiFi"),
    ("portatil_cargador_siempre", "Portátil conectado al cargador permanentemente.", "Computador portátil"),
    ("microondas_recalentados_largos", "Recalentar por tiempos mayores a lo necesario.", "Horno microondas"),
    ("ducha_larga", "Duchas prolongadas (>8 min).", "Ducha eléctrica/instantáneo"),
    ("lavadoras_parciales", "Lavar con cargas parciales frecuentes.", "Lavadora (ciclo activo)"),
]

df_devices = pd.DataFrame(DATA_DEVICES, columns=[
    "device_name","power_w","baseline_hours_per_day","kwh_day_baseline",
    "kwh_day_high_threshold","measure_channel","suggested_actions"
])
df_devices["key"] = df_devices["device_name"].str.lower()
df_habits = pd.DataFrame(DATA_HABITS, columns=["habit_code","description","target_devices"])

def dev_key(nombre:str)->str: return nombre.strip().lower()
def get_device_row(df, nombre):
    row = df[df["key"] == dev_key(nombre)]
    if row.empty: raise KeyError(f"Dispositivo no encontrado: {nombre}")
    return row.iloc[0]
def threshold_day(df, nombre): return float(get_device_row(df, nombre)["kwh_day_high_threshold"])
def baseline_day(df, nombre): return float(get_device_row(df, nombre)["kwh_day_baseline"])
def suggested_action(df, nombre): return str(get_device_row(df, nombre).get("suggested_actions", ""))


In [ ]:
# --- Hechos base ---
class Dispositivo(Fact): pass
class Habito(Fact): pass
class Medicion(Fact): pass
class Hipotesis(Fact): pass
class Accion(Fact): pass


In [ ]:
# --- Motor de inferencia ---
class SistemaEnergia(KnowledgeEngine):
    @Rule(Dispositivo(tipo='aire_acondicionado'),
          Medicion(dispositivo='aire_acondicionado', kwh_periodo=MATCH.k, periodo=MATCH.p),
          TEST(lambda k, p: (k if p=='dia' else k*4) > threshold_day(df_devices, 'Aire acondicionado 12000 BTU')),
          salience=90)
    def ac_sobreconsumo(self, k, p):
        print(f"ACTIVADA: ac_sobreconsumo | periodo={p} | kwh={k:.2f}")
        self.declare(Hipotesis(motivo='sobreconsumo_clima', severidad='alta'))
        self.declare(Accion(texto=suggested_action(df_devices, 'Aire acondicionado 12000 BTU'),
                            dispositivo_objetivo='aire_acondicionado'))

    @Rule(Habito(codigo='ac_puertas_abiertas', activo=True),
          Hipotesis(motivo='sobreconsumo_clima', severidad='alta'),
          salience=95)
    def ac_habito_empeora(self):
        print("ACTIVADA: ac_habito_empeora | hábito=ac_puertas_abiertas -> severidad=crítica")
        for fid, fact in list(self.facts.items()):
            if isinstance(fact, Hipotesis) and fact['motivo']=='sobreconsumo_clima':
                self.retract(fid); break
        self.declare(Hipotesis(motivo='sobreconsumo_clima', severidad='critica'))

    @Rule(Dispositivo(tipo='iluminacion'),
          Habito(codigo='luces_encendidas_sin_uso', activo=True),
          Medicion(dispositivo='iluminacion', kwh_periodo=MATCH.k, periodo='dia'),
          TEST(lambda k: k > baseline_day(df_devices, 'Iluminación LED x10')),
          salience=60)
    def luces_ineficientes(self, k):
        print(f"ACTIVADA: luces_ineficientes | kwh_dia={k:.2f}")
        self.declare(Hipotesis(motivo='iluminacion_ineficiente', severidad='media'))
        self.declare(Accion(texto=suggested_action(df_devices, 'Iluminación LED x10'),
                            dispositivo_objetivo='iluminacion'))

    @Rule(Dispositivo(tipo='nevera'),
          Medicion(dispositivo='nevera', kwh_periodo=MATCH.k, periodo='dia'),
          TEST(lambda k: k > threshold_day(df_devices, 'Nevera (Inverter)')),
          salience=70)
    def nevera_ajuste(self, k):
        print(f"ACTIVADA: nevera_ajuste | kwh_dia={k:.2f}")
        self.declare(Hipotesis(motivo='ajuste_temp_nevera', severidad='media'))
        self.declare(Accion(texto=suggested_action(df_devices, 'Nevera (Inverter)'),
                            dispositivo_objetivo='nevera'))

    @Rule(OR(
            AND(Hipotesis(motivo='sobreconsumo_clima', severidad=MATCH.s),
                Medicion(dispositivo='aire_acondicionado', kwh_periodo=MATCH.k1, periodo='dia'),
                TEST(lambda k1: k1 <= baseline_day(df_devices, 'Aire acondicionado 12000 BTU'))),
            AND(Hipotesis(motivo='iluminacion_ineficiente', severidad=MATCH.s),
                Medicion(dispositivo='iluminacion', kwh_periodo=MATCH.k2, periodo='dia'),
                TEST(lambda k2: k2 <= baseline_day(df_devices, 'Iluminación LED x10'))),
            AND(Hipotesis(motivo='ajuste_temp_nevera', severidad=MATCH.s),
                Medicion(dispositivo='nevera', kwh_periodo=MATCH.k3, periodo='dia'),
                TEST(lambda k3: k3 <= baseline_day(df_devices, 'Nevera (Inverter)')))
          ),
          salience=100)
    def normalizacion(self, s, k1=None, k2=None, k3=None):
        print("ACTIVADA: normalizacion | consumo <= baseline -> retractar hipótesis y declarar consumo_normalizado")
        for fid, fact in list(self.facts.items()):
            if isinstance(fact, Hipotesis) and fact.get('motivo') != 'consumo_normalizado':
                self.retract(fid); break
        self.declare(Hipotesis(motivo='consumo_normalizado', severidad='baja'))


In [ ]:
# --- Declaración base ---
def declarar_base(engine: KnowledgeEngine):
    engine.declare(Dispositivo(tipo='aire_acondicionado', estado='on', potencia=1200, kwh_dia_actual=None))
    engine.declare(Dispositivo(tipo='iluminacion', estado='on', potencia=90, kwh_dia_actual=None))
    engine.declare(Dispositivo(tipo='nevera', estado='on', potencia=120, kwh_dia_actual=None))
    engine.declare(Dispositivo(tipo='lavadora', estado='off', potencia=500, kwh_dia_actual=None))
    engine.declare(Dispositivo(tipo='router', estado='on', potencia=10, kwh_dia_actual=None))

    engine.declare(Habito(codigo='ac_puertas_abiertas', activo=True, dispositivo_objetivo='aire_acondicionado'))
    engine.declare(Habito(codigo='luces_encendidas_sin_uso', activo=True, dispositivo_objetivo='iluminacion'))
    engine.declare(Habito(codigo='nevera_mala_config', activo=False, dispositivo_objetivo='nevera'))


In [ ]:
# Escenarios A/B/C (ejecución interactiva y muestra de hipótesis/acciones)
engine = SistemaEnergia(); engine.reset(); declarar_base(engine)
engine.declare(Medicion(dispositivo='aire_acondicionado', kwh_periodo=9.8, periodo='dia'))
engine.declare(Medicion(dispositivo='iluminacion', kwh_periodo=baseline_day(df_devices,'Iluminación LED x10')+0.10, periodo='dia'))
engine.declare(Medicion(dispositivo='nevera', kwh_periodo=baseline_day(df_devices,'Nevera (Inverter)')-0.20, periodo='dia'))
print("\n--- RUN: Escenario A ---"); engine.run()

engine_B = SistemaEnergia(); engine_B.reset(); declarar_base(engine_B)
k_ilum_alto = baseline_day(df_devices, 'Iluminación LED x10') + 0.20
engine_B.declare(Medicion(dispositivo='iluminacion', kwh_periodo=k_ilum_alto, periodo='dia'))
engine_B.declare(Medicion(dispositivo='aire_acondicionado', kwh_periodo=baseline_day(df_devices,'Aire acondicionado 12000 BTU')-1.0, periodo='dia'))
engine_B.declare(Medicion(dispositivo='nevera', kwh_periodo=baseline_day(df_devices,'Nevera (Inverter)')-0.10, periodo='dia'))
print("\n--- RUN: Escenario B ---"); engine_B.run()

engine_C = SistemaEnergia(); engine_C.reset(); declarar_base(engine_C)
k_ilum_sobre = baseline_day(df_devices, 'Iluminación LED x10') + 0.30
engine_C.declare(Medicion(dispositivo='iluminacion', kwh_periodo=k_ilum_sobre, periodo='dia'))
engine_C.declare(Medicion(dispositivo='aire_acondicionado', kwh_periodo=baseline_day(df_devices,'Aire acondicionado 12000 BTU')-1.0, periodo='dia'))
engine_C.declare(Medicion(dispositivo='nevera', kwh_periodo=baseline_day(df_devices,'Nevera (Inverter)')-0.10, periodo='dia'))
print("\n--- RUN: Escenario C (antes de acción) ---"); engine_C.run()

engine_C.reset(); declarar_base(engine_C)
k_ilum_normal = baseline_day(df_devices, 'Iluminación LED x10')
engine_C.declare(Medicion(dispositivo='iluminacion', kwh_periodo=k_ilum_normal, periodo='dia'))
engine_C.declare(Medicion(dispositivo='aire_acondicionado', kwh_periodo=baseline_day(df_devices,'Aire acondicionado 12000 BTU')-1.0, periodo='dia'))
engine_C.declare(Medicion(dispositivo='nevera', kwh_periodo=baseline_day(df_devices,'Nevera (Inverter)')-0.10, periodo='dia'))
print("\n--- RUN: Escenario C (después de acción) ---"); engine_C.run()

hipotesis_A = [f for f in engine.facts.values() if isinstance(f, Hipotesis)]
acciones_A = [f for f in engine.facts.values() if isinstance(f, Accion)]
hipotesis_B = [f for f in engine_B.facts.values() if isinstance(f, Hipotesis)]
acciones_B = [f for f in engine_B.facts.values() if isinstance(f, Accion)]
hipotesis_C = [f for f in engine_C.facts.values() if isinstance(f, Hipotesis)]
acciones_C = [f for f in engine_C.facts.values() if isinstance(f, Accion)]

hipotesis_A, acciones_A, hipotesis_B, acciones_B, hipotesis_C, acciones_C



--- RUN: Escenario A ---
ACTIVADA: ac_sobreconsumo | periodo=dia | kwh=9.80
ACTIVADA: ac_habito_empeora | hábito=ac_puertas_abiertas -> severidad=crítica
ACTIVADA: luces_ineficientes | kwh_dia=0.46

--- RUN: Escenario B ---
ACTIVADA: luces_ineficientes | kwh_dia=0.56

--- RUN: Escenario C (antes de acción) ---
ACTIVADA: luces_ineficientes | kwh_dia=0.66

--- RUN: Escenario C (después de acción) ---


([Hipotesis(motivo='sobreconsumo_clima', severidad='critica'),
  Hipotesis(motivo='iluminacion_ineficiente', severidad='media')],
 [Accion(texto='Cerrar puertas/ventanas; setpoint 23–25 °C; limpiar filtros', dispositivo_objetivo='aire_acondicionado'),
  Accion(texto='Sensores o política de apagado; bombillos 5–7 W', dispositivo_objetivo='iluminacion')],
 [Hipotesis(motivo='iluminacion_ineficiente', severidad='media')],
 [Accion(texto='Sensores o política de apagado; bombillos 5–7 W', dispositivo_objetivo='iluminacion')],
 [],
 [])

In [ ]:
# --- RESULTADOS (para el autograder) ---
def resumen_facts(engine):
    hips = [dict(f) for f in engine.facts.values() if isinstance(f, Hipotesis)]
    accs = [dict(f) for f in engine.facts.values() if isinstance(f, Accion)]
    return {"hipotesis": hips, "acciones": accs}

def _run_A():
    e = SistemaEnergia(); e.reset(); declarar_base(e)
    e.declare(Medicion(dispositivo='aire_acondicionado', kwh_periodo=9.8, periodo='dia'))
    e.run(); return resumen_facts(e)

def _run_B():
    e = SistemaEnergia(); e.reset(); declarar_base(e)
    e.declare(Medicion(dispositivo='iluminacion', kwh_periodo=baseline_day(df_devices,'Iluminación LED x10')+0.20, periodo='dia'))
    e.run(); return resumen_facts(e)

def _run_C():
    e = SistemaEnergia(); e.reset(); declarar_base(e)
    # Primero simula un sobreconsumo
    e.declare(Medicion(dispositivo='iluminacion', kwh_periodo=baseline_day(df_devices,'Iluminación LED x10')+0.3, periodo='dia'))
    e.run()  # genera la hipótesis ilumiación_ineficiente

    # Luego, normaliza el consumo
    e.reset(); declarar_base(e)
    e.declare(Medicion(dispositivo='iluminacion', kwh_periodo=baseline_day(df_devices,'Iluminación LED x10'), periodo='dia'))
    e.run()
    return resumen_facts(e)


RESULTADOS = {"A": _run_A(), "B": _run_B(), "C": _run_C()}
RESULTADOS


ACTIVADA: ac_sobreconsumo | periodo=dia | kwh=9.80
ACTIVADA: ac_habito_empeora | hábito=ac_puertas_abiertas -> severidad=crítica
ACTIVADA: luces_ineficientes | kwh_dia=0.56
ACTIVADA: luces_ineficientes | kwh_dia=0.66


{'A': {'hipotesis': [{'motivo': 'sobreconsumo_clima',
    'severidad': 'critica',
    '__factid__': 12}],
  'acciones': [{'texto': 'Cerrar puertas/ventanas; setpoint 23–25 °C; limpiar filtros',
    'dispositivo_objetivo': 'aire_acondicionado',
    '__factid__': 11}]},
 'B': {'hipotesis': [{'motivo': 'iluminacion_ineficiente',
    'severidad': 'media',
    '__factid__': 10}],
  'acciones': [{'texto': 'Sensores o política de apagado; bombillos 5–7 W',
    'dispositivo_objetivo': 'iluminacion',
    '__factid__': 11}]},
 'C': {'hipotesis': [], 'acciones': []}}